# Apply NLP to PRS Data

### Import Libaries

In [2]:
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Load PRS Data

In [3]:
data = pd.read_csv('prs_qboa_survey_data.csv')
print(data.shape)
data.head()

(378491, 21)


,company_id,auth_id,survey_date,prs_qboa_raw,prs_qboa_segment,prs_qboa_score,prs_qboa_voice,prs_qbo_raw,prs_qbo_segment,prs_qbo_score,...,qboa_company_name,qboa_channel,qboa_signup_date,country,company_tenure,wholesale_flag,proadvisor_flag,proadvisor_tier,qbo_certification_flag,qbse_attached_flag
0,1325779545,127088051,2015-04-01,0,Detractor,-1,This new platform sucks. I am unable to manage...,0,Detractor,-1,...,"TaxWorks Plus, Inc",Unknown,2015-01-09,United States,3 Months or Less,Wholesale,ProAdvisor,ELITE,Not Certified,QBSE Not Attached
1,1345924720,1346476015,2015-04-01,2,Detractor,-1,still too many glitches,2,Detractor,-1,...,Claire Prescott,Client Invite,2015-02-10,United States,3 Months or Less,Not Wholesale,ProAdvisor,SILVER,Not Certified,QBSE Not Attached
2,1310633145,300822642,2015-04-01,5,Detractor,-1,jjjjjjjjjjjjjjjjjjjjjjj,4,Detractor,-1,...,"Hamilton, Stuebi & Young, LLC",Unknown,2014-12-05,United States,4 - 12 Months,Not Wholesale,Not ProAdvisor,Not ProAdvisor,Not Certified,QBSE Not Attached
3,1310925660,1291056130,2015-04-01,10,Promoter,1,works so well,10,Promoter,1,...,Damon Felix Fanucchi,Client Invite,2014-12-09,United States,4 - 12 Months,Not Wholesale,ProAdvisor,SILVER,Not Certified,QBSE Not Attached
4,1219923465,181802120,2015-04-01,0,Detractor,-1,"It is slow, difficult to use, challenging to f...",0,Detractor,-1,...,donnaharris@simplifymybooks.com's company,Unknown,2014-06-03,United States,4 - 12 Months,Not Wholesale,Not ProAdvisor,Not ProAdvisor,Not Certified,QBSE Not Attached


In [4]:
data.tail()

,company_id,auth_id,survey_date,prs_qboa_raw,prs_qboa_segment,prs_qboa_score,prs_qboa_voice,prs_qbo_raw,prs_qbo_segment,prs_qbo_score,...,qboa_company_name,qboa_channel,qboa_signup_date,country,company_tenure,wholesale_flag,proadvisor_flag,proadvisor_tier,qbo_certification_flag,qbse_attached_flag
378486,404573131,569506960,2019-08-10,5,Detractor,-1,NaN,6,Detractor,-1,...,pamlim.ea@gmail.com's company-QB Accountant,Migrated,2013-12-10,United States,12 Months or More,Wholesale,ProAdvisor,GOLD,Not Certified,QBSE Not Attached
378487,123146092739979,193514820994419,2019-08-10,10,Promoter,1,NaN,7,Passive,0,...,"Davis & Snead, CPAs",Client Invite,2018-08-21,United States,4 - 12 Months,Not Wholesale,ProAdvisor,GOLD,Certified,QBSE Not Attached
378488,123145799964739,123145800920584,2019-08-10,8,Passive,0,NaN,9,Promoter,1,...,NaN,Client Invite,2017-04-09,United States,12 Months or More,Not Wholesale,ProAdvisor,SILVER,Not Certified,QBSE Not Attached
378489,571270615,111089839,2019-08-10,0,Detractor,-1,NaN,9,Promoter,1,...,karol@morerafinance.com's Company,Unknown,2013-01-11,United States,12 Months or More,Wholesale,ProAdvisor,PLATINUM,Not Certified,QBSE Not Attached
378490,1297070835,193514332782902,2019-08-10,10,Promoter,1,NaN,10,Promoter,1,...,"Accounting Gurus, LLC",Client Invite,2014-10-29,United States,12 Months or More,Wholesale,ProAdvisor,ELITE,Not Certified,QBSE Attached


### Text Preprocessing

Data processing involves the following steps:

1. Remove empty texts
2. Extract relevant columns (text reviews, ratings)
3. Convert 11 classes into two classes (positive = 1 and negative = 0)
4. Convert words to lower case
5. Remove stop words
6. Remove punctuation from texts
7. Stemming or Lemmatization

Note the missing data in the reviews.

Extract reviews and ratings, remove records with empty strings:

In [5]:
data=data[~data['prs_qboa_voice'].isnull()]
text=data['prs_qboa_voice']
rating=data['prs_qboa_raw']
print('Remaining rows: ',text.shape[0])
print("Sample reviews:\n",text.head())
print("\nCorresponding ratings:\n",rating[:5])

Remaining rows:  154429
Sample reviews:
 0    This new platform sucks. I am unable to manage...
1                              still too many glitches
2                              jjjjjjjjjjjjjjjjjjjjjjj
3                                        works so well
4    It is slow, difficult to use, challenging to f...
Name: prs_qboa_voice, dtype: object

Corresponding ratings:
 0     0
1     2
2     5
3    10
4     0
Name: prs_qboa_raw, dtype: int64


### Simplify Ratings to 0 (for 0-7) or 1 (for 8-10)

In [7]:
ratings = [int(rate>7) for rate in rating]
print(ratings[:5])
pd.DataFrame(ratings)[0].value_counts()

[0, 0, 0, 1, 0]


1    86229
0    68200
Name: 0, dtype: int64

### Define function to determine how many unique words in corpus:

In [8]:
def count_words(text_list):
    cv = CountVectorizer(binary=True)
    cv.fit(text_list)
    X = cv.transform(text_list)
    return(X.shape[1])

In [9]:
print('Number of unique words: ',count_words(text))
text.head()

Number of unique words:  26953


0    This new platform sucks. I am unable to manage...
1                              still too many glitches
2                              jjjjjjjjjjjjjjjjjjjjjjj
3                                        works so well
4    It is slow, difficult to use, challenging to f...
Name: prs_qboa_voice, dtype: object

### Remove Stop Words

In [10]:
stop = stopwords.words('english') # Set checking is faster in Python than list.
print(len(stop),'stop words:\n', stop)

# List comprehension.
text_nosw = text.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop)]))

#print('\nNumber of unique words: ',count_words(text_nosw))
print('\nReviews with no stop words:\n',text_nosw[:5])

179 stop words:
 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same'

### Define function (and call it) to remove punctuation and convert to all lowercase:

In [11]:
REPLACE_WITH_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\<)|(\>)")
REPLACE_NO_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|(\_)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

In [12]:
text_clean = pd.DataFrame(preprocess_reviews(text_nosw),columns=['text'])
print('Number of unique words: ',count_words(text_clean['text']))
text_clean.head()

Number of unique words:  28224


,text
0,new platform sucks unable manage clients look...
1,still many glitches
2,jjjjjjjjjjjjjjjjjjjjjjj
3,works well
4,slow difficult use challenging figure accomp...


In [13]:
print(text[0])
print(text[14])

This new platform sucks. I am unable to manage my clients and look like a fool in from my client that I tried to convert to in 2015 with the 9.99 promotion.  I am unable to access and support did nothing for me.
printing reports&#x2F;no window available&#x2F;cant save to local computer


In [14]:
print(text_clean['text'][0])
print(text_clean['text'][9])

new platform sucks  unable manage clients look like fool client tried convert 2015 9 99 promotion  unable access support nothing me 
printing reports&#x2f no window available&#x2f cant save local computer


### Apply Porter Stemmer

In [15]:
porter=PorterStemmer()

text_clean_stemmed = []
for reviews in text_clean['text']:
    stems = [porter.stem(word)+' ' for word in reviews.split()]
    text_clean_stemmed.append("".join(stems))

print('Number of unique words: ',count_words(text_clean_stemmed))    
text_clean_stemmed[:5]

Number of unique words:  21701


['new platform suck unabl manag client look like fool client tri convert 2015 9 99 promot unabl access support noth me ',
 'still mani glitch ',
 'jjjjjjjjjjjjjjjjjjjjjjj ',
 'work well ',
 'slow difficult use challeng figur accomplish simplest task quickbook desktop far superior product qbo ridicul ']

### Apply Lemmatization

In [16]:
lemmatizer =WordNetLemmatizer()

text_clean_lemmatized = []
for reviews in text_clean['text']:
    stems = [lemmatizer.lemmatize(word)+' ' for word in reviews.split()]
    text_clean_lemmatized.append("".join(stems))

print('Number of unique words: ',count_words(text_clean_lemmatized))    
text_clean_lemmatized[:5]

Number of unique words:  26573


['new platform suck unable manage client look like fool client tried convert 2015 9 99 promotion unable access support nothing me ',
 'still many glitch ',
 'jjjjjjjjjjjjjjjjjjjjjjj ',
 'work well ',
 'slow difficult use challenging figure accomplish simplest task quickbooks desktop far superior product qbo ridiculous ']

## Build Model to predict Sentiment

The following steps are used in creating Sentiment model:

1. Count Vectorization
2. TF-IDF Vectorization
3. Split Training Data
4. Build Logistic Regression Model
5. Parameter Tuning
6. Create final model
7. Surmise sentiment words

### Vectorize reviews using Count Vectorization

In [17]:
cv = CountVectorizer(binary=True)
X = cv.fit_transform(text_clean_stemmed)

print('Size of X: ',X.shape)
print('First review: \n',X[0])
print('Sample features: \n',cv.get_feature_names()[10900:11000])

Size of X:  (154429, 21701)
First review: 
   (0, 10901)	1
  (0, 12966)	1
  (0, 18465)	1
  (0, 783)	1
  (0, 15188)	1
  (0, 657)	1
  (0, 228)	1
  (0, 4347)	1
  (0, 19536)	1
  (0, 7346)	1
  (0, 10351)	1
  (0, 10570)	1
  (0, 3600)	1
  (0, 10743)	1
  (0, 19744)	1
  (0, 18378)	1
  (0, 14560)	1
  (0, 12005)	1
Sample features: 
 ['mdt', 'me', 'meal', 'mean', 'meani', 'meaning', 'meaningless', 'meant', 'meantim', 'meanwhil', 'measur', 'meat', 'meatier', 'meaur', 'mechan', 'med', 'meddl', 'media', 'mediamdevic', 'medic', 'medicar', 'medicor', 'mediev', 'medio', 'mediocor', 'mediocr', 'mediocrenreport', 'mediorcenshort', 'medium', 'mediumspeedbookswithquirkyidiosyncrasi', 'medley', 'medocr', 'medssi', 'meed', 'meer', 'mees', 'meet', 'meeting', 'meg', 'meh', 'meilleur', 'mejor', 'mejorando', 'meleni', 'melika', 'member', 'membership', 'meme', 'memeb', 'memo', 'memor', 'memori', 'memoris', 'memorize', 'memorized', 'men', 'menand', 'meni', 'menial', 'menlov', 'mennonit', 'meno', 'menori', 'mensuel'

### Vectorize reviews using TFIDF Vectorizer

Let's define a few terms related to TF-IDF:

**TF (Term Frequency)** :
Denotes the contribution of the word to the document i.e. words relevant to the document should be frequent. 

            TF = (Number of times term t appears in a document)/(Number of terms in the document)

**IDF (Inverse Document Frequency)** :
If a word has appeared in all the documents, then probably that word is not relevant to a particular document. 
But, if it has appeared in a subset of documents then probably the word is of some relevance to the documents it is present in.

           IDF = log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.

In [18]:
tfidf = TfidfVectorizer() #ngram_range=(1, 2))
X = tfidf.fit_transform(text_clean_stemmed)

print('Size of X: ',X.shape)
print('First review: \n',X[0])
print('Sample features: \n',tfidf.get_feature_names()[10900:11000])

Size of X:  (154429, 21701)
First review: 
   (0, 12005)	0.16997032066293502
  (0, 14560)	0.19819278336914592
  (0, 18378)	0.2079454023503958
  (0, 19744)	0.45511347221140813
  (0, 10743)	0.1773455019956885
  (0, 3600)	0.18686293619824967
  (0, 10570)	0.1767288901897543
  (0, 10351)	0.11350651207087983
  (0, 7346)	0.3333941432442282
  (0, 19536)	0.17373470348050826
  (0, 4347)	0.2403539351351772
  (0, 228)	0.29919209675070857
  (0, 657)	0.3100586062637267
  (0, 15188)	0.25960084137277506
  (0, 783)	0.11590455597314346
  (0, 18465)	0.13639195508149998
  (0, 12966)	0.2127511671866517
  (0, 10901)	0.20377482968442318
Sample features: 
 ['mdt', 'me', 'meal', 'mean', 'meani', 'meaning', 'meaningless', 'meant', 'meantim', 'meanwhil', 'measur', 'meat', 'meatier', 'meaur', 'mechan', 'med', 'meddl', 'media', 'mediamdevic', 'medic', 'medicar', 'medicor', 'mediev', 'medio', 'mediocor', 'mediocr', 'mediocrenreport', 'mediorcenshort', 'medium', 'mediumspeedbookswithquirkyidiosyncrasi', 'medley', 'm

In [19]:
tfidf_results = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
tfidf_results = pd.DataFrame(columns=['tfidf']).from_dict(
                    dict(tfidf_results), orient='index')
tfidf_results.columns = ['tfidf']
tfidf_results.sort_values(by=['tfidf'], ascending=False).head(10)

,tfidf
ふぇうぃｂ,12.254349
reqest,12.254349
requirementsn,12.254349
impercept,12.254349
imperfectionsng,12.254349
requikrements,12.254349
imping,12.254349
implant,12.254349
implel,12.254349
requestng,12.254349


In [20]:
tfidf_results.sort_values(by=['tfidf'], ascending=True).head(10)

,tfidf
use,2.329296
client,2.775242
easi,2.787817
account,3.002675
desktop,3.089163
version,3.093304
amp,3.108334
onlin,3.176683
x27,3.331757
like,3.371541


### Split data 75-25

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, ratings, train_size = 0.75, random_state=1)

### Split training data for parameter tuning

In [22]:
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, y_train, train_size = 0.75, random_state=1)

for c in [0.01, 0.05, 0.25, 0.5, 1, 2, 4, 8, 10]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train2, y_train2)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.8143389970990468
Accuracy for C=0.05: 0.8306741262605333
Accuracy for C=0.25: 0.838168255283879
Accuracy for C=0.5: 0.8395151263986739
Accuracy for C=1: 0.839687802182622
Accuracy for C=2: 0.8390316342036193
Accuracy for C=4: 0.8373394115209283
Accuracy for C=8: 0.83540544274071
Accuracy for C=10: 0.8348183450752866


### Train/Test model using best parameter on all of data

In [23]:
final_model = LogisticRegression(C=1)
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model.predict(X_test)))

Final Accuracy: 0.8414059262329051


### What are the coefficients?

In [24]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
    
features=pd.DataFrame(list(feature_to_coef.items()), columns=['Features','Coefficients'])
features.head(10)

,Features,Coefficients
0,00,-0.416035
1,000,-0.749372
2,00000,0.364080
3,0001,0.000000
4,000th,-0.129536
5,000x,-0.050742
6,01,-0.069853
7,0113,-0.192663
8,0151,0.000000
9,01nit,0.336213


### What are the most positive words?

In [25]:
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:15]:
    print (best_positive)

('love', 4.398575755498221)
('awesom', 3.99392143120901)
('great', 3.701669165398693)
('excel', 3.591794023989809)
('eas', 3.3076066438632425)
('easi', 3.2675227640456077)
('best', 2.9614427809927495)
('amaz', 2.862395673535808)
('10', 2.842353426389865)
('wherev', 2.8218055495939196)
('perfect', 2.6629086905679946)
('tool', 2.6384798159933958)
('save', 2.4900086328417306)
('anywher', 2.472131836852879)
('futur', 2.4520819477098956)


### What are the most negative words?

In [26]:
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:15]:
    print (best_negative)

('poor', -5.772262104278763)
('difficult', -5.751942704953135)
('slow', -5.497332847986067)
('terribl', -4.949237060697209)
('clunki', -4.526206844451933)
('imposs', -4.525519761910107)
('hard', -4.45445059222422)
('lack', -4.425420643329863)
('worst', -4.313577315831901)
('don', -4.23081036078803)
('doesn', -4.220828352890409)
('cannot', -4.177126602614009)
('horribl', -4.141687054031067)
('desktop', -4.10451832536255)
('cumbersom', -3.9665690354344254)
